In [1]:
from __future__ import print_function

import os;
import shutil;
import codecs;
import zipfile;

import re;
import requests
from lxml import html
from string import ascii_uppercase;
import chardet;

import random;

In [2]:
from os.path import expanduser
home = expanduser("~")
home

'/Users/joseeleandrocustodio'

In [3]:
baseAddress = 'https://www.letras.mus.br';

# Utility Functions

In [43]:
def slugify(value):
    """
    Normalizes string, converts to lowercase, removes non-alpha characters,
    and converts spaces to hyphens.
    """
    import unicodedata
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore')
    value = unicode(re.sub('[^\w\s-]', '', value).strip().lower())
    value = unicode(re.sub('[-\s]+', '-', value))
    return value;

In [ ]:
#use chardet to detect the actual encoding of the content and than encode it.
def decode(content):
    enconding = chardet.detect(content)['encoding']; #converting to the right encoding
    content = content.decode(enconding, errors='ignore');
    return content;

# Web Scrapping Related Functions

In [ ]:
def listArtistByLetter(letter):
    page = requests.get(baseAddress+'/letra/'+letter+'/artistas.html')
    tree = html.fromstring(page.content);
    return [
        a.attrib['href'] for a in tree.xpath('//div[@class="artistas-a js-alphabet-cnt"]/ul/li//a')
    ];

In [44]:
def listSongsFromArtist(artist):
    page = requests.get(baseAddress+artist)
    tree = html.fromstring(page.content);
    r = [a.attrib['href']+"|"+a.text_content() for a in tree.xpath('//div[@class="cnt-list--alp"]/li//a')];
    r +=[a.attrib['href']+"|"+a.text_content() for a in tree.xpath('//*[@id="cnt_top"]//li//a')];
    
    r = list(set(r)); #removing duplicates
    return r;  

In [ ]:
# retrive only one song lyrics
def getLyrics(id):
    return requests.get(baseAddress+id).content;

# Reading all artists into array to futher processing

## Reading the whole artist list and saving it into a file

In [4]:
artists = [];

for letter in ascii_uppercase:
    artists += listArtistByLetter(letter)
    
with open(home+'/letras/artists.txt','w') as f:
    f.writelines("%s\n" % l for l in artists);

In [5]:
len(artists)

214346

## Reading only the most visited artist (probably it contains the weekly report, not the all times report)

In [61]:
maisAcessados = baseAddress+'/mais-acessadas/';
maisAcessados = requests.get(maisAcessados);
maisAcessados = html.fromstring(maisAcessados.content);
maisAcessados = [a.attrib['href'] for a in maisAcessados.xpath('//*[@class="top-list_art"]/li//a')]

In [62]:
len(maisAcessados)

302

In [42]:
maisAcessados[0:10]

['/1kilo/',
 '/matheus-kauan/',
 '/henrique-e-juliano/',
 '/jojo-maronttinni/',
 '/harpa-crista/',
 '/jorge-mateus/',
 '/ed-sheeran/',
 '/pineapple/',
 '/gabriela-rocha/',
 '/anavitoria/']

## reading the previously saved artists list  from file

In [45]:
artists = []
with open(home+'/letras/artists.txt','r') as f:
    artists = f.readlines();

In [46]:
artists[0].strip()

'/anavitoria/'

In [47]:
random.shuffle(artists);
artists[0].strip()

'/elcio-oliveira-e-banda/'

In [63]:
def downloadArtist(currentArtist):
        
    songList = listSongsFromArtist(currentArtist[0:-1]);
    
    if len(songList) <3:
        return;

    artistDir= home + "/letras"+currentArtist[0:-1]+".zip";
    
    if os.path.exists(artistDir):
        return;
    
    thezip = zipfile.ZipFile(artistDir,"w")
    
    savedLyrics = 0;
    inZip = []; #adicional check for duplicated files.
    for s in songList:
        s = s.strip().split("|");
        
        fileName = slugify(unicode(s[1])) + ".html";
        #print (fileName);
        if fileName in inZip:
            continue;
        
        content = decode(getLyrics(s[0]));

        thezip.writestr(fileName, content.encode("UTF-8"));
        inZip.append(fileName);
    
    thezip.close();

### maisAcessadas

In [64]:
print ("Faltando %d artistas para ler " % len(maisAcessados))

while len(maisAcessados) >0:
    currentArtist = maisAcessados[0].strip();
    print (currentArtist);
    downloadArtist(currentArtist);
    
    maisAcessados.remove(maisAcessados[0]);
    if maisAcessados[0] in  artists:
        artists.remove(maisAcessados[0]);

Faltando 302 artistas para ler 
/1kilo/
/matheus-kauan/
/henrique-e-juliano/
/jojo-maronttinni/
/harpa-crista/
/jorge-mateus/
/ed-sheeran/
/pineapple/
/gabriela-rocha/
/anavitoria/
/marilia-mendonca/
/gusttavo-lima/
/legiao-urbana/
/anitta/
/midian-lima/
/aline-barros/
/ferrugem/
/bangtan-boys/
/camila-cabello/
/charlie-brown-jr/
/marchinhas-de-carnaval/
/fernandinho/
/projota/
/anderson-freire/
/bruna-karla/
/wesley-safadao/
/thiago-brava/
/gaab/
/bruno-e-marrone/
/caetano-veloso/
/fernanda-brum/
/lana-del-rey/
/hungria-hip-hop/
/ministerio-zoe/
/chico-buarque/
/pabllo-vittar/
/diante-do-trono/
/kemilly-santos/
/gustavo-mioto/
/racionais-mcs/
/simone-simaria-as-coleguinhas/
/damares/
/coldplay/
/maneva/
/dua-lipa/
/lucas-lucco/
/luan-santana/
/bruno-mars/
/demi-lovato/
/felipe-araujo/
/eyshila/
/cassiane/
/imagine-dragons/
/shirley-carvalhaes/
/ze-neto-cristiano/
/the-beatles/
/beyonce/
/justin-bieber/
/rose-nascimento/
/rihanna/
/voz-da-verdade/


KeyboardInterrupt: 

In [ ]:
updateSongs = False;
count = 30

print ("Faltando %d artistas para ler " % len(artists))

while len(artists) >0 and count > 0:
    #avoiding being blocked
    count -= 1;
    
    #picking and removing 1 artist at time
    currentArtist = artists[0].strip();
    artists.remove(artists[0]);
    downloadArtist(artists);